In [1]:
import sqlalchemy as db
import pandas as pd
from sqlalchemy import extract, func

In [2]:
sqlite_URL='sqlite:///database.db'
engine = db.create_engine(sqlite_URL)
connection = engine.connect()
metadata = db.MetaData()
students = db.Table('Student', metadata, autoload_with=engine)

In [3]:
# Print the column names
print(students.columns.keys())

['id', 'firstname', 'lastname', 'email', 'age', 'created_at', 'bio']


In [4]:
# Print full table metadata
print(repr(metadata.tables['Student']))

Table('Student', MetaData(), Column('id', INTEGER(), table=<Student>, nullable=False), Column('firstname', VARCHAR(length=100), table=<Student>, nullable=False), Column('lastname', VARCHAR(length=100), table=<Student>, nullable=False), Column('email', VARCHAR(length=80), table=<Student>, nullable=False), Column('age', INTEGER(), table=<Student>), Column('created_at', DATETIME(), table=<Student>, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x0000020BBB41C850>, for_update=False)), Column('bio', TEXT(), table=<Student>), schema=None)


In [5]:
# Equivalent to 'SELECT * FROM Student'
query = db.select(students)
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ResultSet

[(1, 'john', 'doe', 'john_doe@example.com', 23, datetime.datetime(2023, 6, 20, 21, 32, 13), 'Biology student... might change major?'),
 (2, 'Sammy', 'Shark', 'sammyshark@example.com', 20, datetime.datetime(2023, 6, 20, 21, 54, 17), 'Marine biology student'),
 (3, 'Carl', 'White', 'carlwhite@example.com', 22, datetime.datetime(2023, 6, 20, 21, 54, 17), 'Marine geology student'),
 (4, 'Seal', 'Davis', 'thaiisdavis@yahoo.com', 22, datetime.datetime(2023, 6, 21, 1, 38, 5), 'Ihave terrible grades :)')]

In [6]:
# Read Student table as a dataframe
students_df = pd.DataFrame(ResultSet)
students_df.columns = students.columns.keys()
students_df

,id,firstname,lastname,email,age,created_at,bio
0,1,john,doe,john_doe@example.com,23,2023-06-20 21:32:13,Biology student... might change major?
1,2,Sammy,Shark,sammyshark@example.com,20,2023-06-20 21:54:17,Marine biology student
2,3,Carl,White,carlwhite@example.com,22,2023-06-20 21:54:17,Marine geology student
3,4,Seal,Davis,thaiisdavis@yahoo.com,22,2023-06-21 01:38:05,Ihave terrible grades :)


In [7]:
# SQL query: SELECT * FROM Student WHERE age=22
query = db.select(students).where(students.columns.age==22)
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ResultSet

[(3, 'Carl', 'White', 'carlwhite@example.com', 22, datetime.datetime(2023, 6, 20, 21, 54, 17), 'Marine geology student'),
 (4, 'Seal', 'Davis', 'thaiisdavis@yahoo.com', 22, datetime.datetime(2023, 6, 21, 1, 38, 5), 'Ihave terrible grades :)')]

In [8]:
# SQL query: SELECT email, age FROM Student WHERE age in (20,23)
query = db.select(students.columns.email, students.columns.age).where(students.columns.age.in_([20,23]))
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ResultSet

[('john_doe@example.com', 23), ('sammyshark@example.com', 20)]

In [9]:
# SQL query: SELECT * FROM Student WHERE id<3 AND NOT age=23
query = db.select(students).where(db.and_(students.columns.id<3, students.columns.age!=23))
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ResultSet

[(2, 'Sammy', 'Shark', 'sammyshark@example.com', 20, datetime.datetime(2023, 6, 20, 21, 54, 17), 'Marine biology student')]

In [10]:
# SQL query: SELECT * FROM Student ORDER BY age DESC
query = db.select(students).order_by(db.desc(students.columns.age))
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ResultSet

[(1, 'john', 'doe', 'john_doe@example.com', 23, datetime.datetime(2023, 6, 20, 21, 32, 13), 'Biology student... might change major?'),
 (3, 'Carl', 'White', 'carlwhite@example.com', 22, datetime.datetime(2023, 6, 20, 21, 54, 17), 'Marine geology student'),
 (4, 'Seal', 'Davis', 'thaiisdavis@yahoo.com', 22, datetime.datetime(2023, 6, 21, 1, 38, 5), 'Ihave terrible grades :)'),
 (2, 'Sammy', 'Shark', 'sammyshark@example.com', 20, datetime.datetime(2023, 6, 20, 21, 54, 17), 'Marine biology student')]

In [11]:
# SQL query: SELECT SUM(age) FROM Student
query = db.select(db.func.sum(students.columns.age))
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ResultSet

[(87,)]

In [12]:
# SQL query: SELECT created_at,AVG(age) FROM Student GROUP BY created_at
query = db.select(db.func.avg(students.columns.age).label('age'),students.columns.created_at).group_by(students.columns.created_at)
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ResultSet

[(23.0, datetime.datetime(2023, 6, 20, 21, 32, 13)),
 (21.0, datetime.datetime(2023, 6, 20, 21, 54, 17)),
 (22.0, datetime.datetime(2023, 6, 21, 1, 38, 5))]

In [13]:
# SQL query: SELECT created_at,AVG(age) FROM Student GROUP BY created_at HAVING day==21
query = db.select(db.func.avg(students.columns.age).label('age'),students.columns.created_at).\
        group_by(students.columns.created_at).\
        having(extract('day', students.columns.created_at) == 21)
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ResultSet

[(22.0, datetime.datetime(2023, 6, 21, 1, 38, 5))]

In [14]:
# SQL query: SELECT * FROM Student WHERE id = (SELECT * FROM Student WHERE age=23)
subquery = db.select(students.columns.id).where(students.columns.age == 23).scalar_subquery()
query = db.select(students).where(students.columns.id==subquery)
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ResultSet

[(1, 'john', 'doe', 'john_doe@example.com', 23, datetime.datetime(2023, 6, 20, 21, 32, 13), 'Biology student... might change major?')]